<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score 
from sklearn.linear_model import LinearRegression,LogisticRegressionCV,LinearRegression,LassoCV,RidgeCV 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection  import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.dummy import DummyClassifier


import warnings 
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.duplicated().sum() 

153

In [5]:
data.isnull().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

**ВЫВОДЫ** 

- пропущенных значений нет
- необходимо изменить тип данных в столбце Возрастиз float в int
- избавиться от 153 дубликатов 

In [6]:
data.drop_duplicates().reset_index(drop = True)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4842,0,28.0,35700.0,2,0
4843,0,34.0,52400.0,1,0
4844,0,20.0,33900.0,2,0
4845,1,22.0,32700.0,3,0


In [7]:
data['Возраст'] = data['Возраст'].astype('int')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   int64  
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 195.4 KB


**ВЫВОДЫ**

- от дубликатов избавились
- к необходимым значениям данные привели

## Умножение матриц

Ответим на вопрос и обоснуем решение.


Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)


- a. Изменится. Приведите примеры матриц.
- b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Разделим признаки на features и target 

In [9]:
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']
print(features.shape, target.shape)

(5000, 4) (5000,)


In [10]:
model  = LinearRegression()
model.fit(features, target)
prediction = model.predict(features)
print(r2_score(target, prediction))


0.42494550286668


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Укажем основные свойства обратной матрицы:
$$
det(A) = \\frac{1}  {det(A)}
$$

$$
(AB)^{-1} =  B^{-1}A^{-1}
$$

$$
(A^T)^{-1} = ((A)^{-1})^T
$$



Для двух квадратных оббратимых матриц $A$ $B$: 
$$
(A^T)^{-1} = ((A)^{-1}))^T
$$


Укажем основыне свойства единичной матрицы :
$$
AE = EA =A
$$

$$
AA^{1}=EE
$$

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится

**Обоснование:** 

Заменим матрицу $X$ на матрицу $Z$: 

$$
Z = XP   (1.1)
$$

$P$  а данном случае - обратная матрица с неизвестными значениями, на котороую может быть умножена матрица X.

Заменим $X$ на $Z$ и вычислим чему будет равно предсказание и вектор весов:
$$
a_1 = Zw_1 (1.2)
$$

$$
w_1 = (Z^T Z)^{-1} Z^T y (1.3)
$$


Подставим в уравнение 1.2 правую часть уравнения 1.3 :

$$
a_1 = Z(Z^T Z)^ {-1} Z^T y  (1.4)
$$


Заменим все $Z$ правой частью уравнения 1.1:
$$
a_1 =XP ((XP)^T (XP))^{-1} (XP)^T y  (1.5)
$$

Для данного этапа нам нам понадобится свойство обратной матрицы:
$$
(AB)^{-1} =  B^{-1}A^{-1}
$$


Раскроем $((XP)^T (XP))^{-1}$ :
$$
а_1 = XP ((XP)^T (XP))^{-1} (XP)^T y = XP(XP)^{-1}((XP)^T)^{-1} (XP)^T y = XPP^{-1}X^{-1}((XP)^T)^{1} (XP)^T (1.6)
$$

Из умножения получится, что $PP^{-1} = E$

Для данного этапа нам нам понадобится свойство транспортированной матрицы:
$$
(AB)^T = B^T A^T 
$$

Умножение на единичную матрицу ничего не поменяет
$$
a_1 = XEX^{-1}((XP)^T)^{-1} (XP)^T y = XX^{-1}(P^T X^T)^{-1} P^T X^T y = XX^{-1} (X^T)^{-1} (P^T)^{-1} P^T X^T y (1.7)
$$

Умножение привет к $(P^T)^{-1} P^T = E$ В итоге из уравнения 1.7 получится :

$$
a_1 = XX^{-1} (X^T)^{-1} E X^T y= X(X^T X)^{-1} X^T y = Xw = a
$$

Отсюда можно сказать, что значение предсказания $a$ не поменяется, если умножить матрицу признаков на обратную матрицу. 

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w'= (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^ {-1} E X^Ty
$$

$$
w' =  P^{-1}w
$$
$$
w' = ((XP)^T(XP))^{-1}(XP)^Ty
$$

Создадим обратную матрицу и умножим для проверки 

In [11]:
random_matrix =  np.matrix(np.random.randint(10, size = (4,4)))
print('Рандоманая матрица:', random_matrix)
print()
reverse_matrix  = np.linalg.inv(random_matrix)
print('Обратная матрица:',reverse_matrix )
features_1 = features.dot(reverse_matrix)
features_1

Рандоманая матрица: [[6 5 6 0]
 [6 8 3 2]
 [5 6 8 8]
 [8 1 6 3]]

Обратная матрица: [[-0.10885246  0.11803279 -0.09639344  0.17836066]
 [ 0.04262295  0.09836066  0.01967213 -0.11803279]
 [ 0.24       -0.2         0.08       -0.08      ]
 [-0.20393443  0.05245902  0.0904918   0.05704918]]


,0,1,2,3
0,11905.434754,-9915.796721,3968.800656,-3972.603934
1,9121.756721,-7595.422951,3040.995410,-3045.372459
2,5041.236066,-4197.147541,1680.570492,-1683.422951
3,10008.487213,-8337.829508,3336.594098,-3338.364590
4,6265.084590,-5217.127869,2088.454426,-2091.126557
...,...,...,...,...
4995,8568.785574,-7137.140984,2856.731803,-2859.190820
4996,12577.245246,-10476.603279,4192.759344,-4195.956066
4997,8136.444590,-6777.927869,2712.574426,-2714.246557
4998,7848.217049,-6537.560656,2616.607869,-2618.247213


def generate_invertible_matrix(size):
    try:
        matrix = np.random.normal(size=(size, size))
        # проверим матрицу на обратимость, если нет, пробуем сгенерировать еще раз
        # таким образом гарантируем, что матрица стопроцентно будет обратимой
        np.linalg.inv(matrix)
    except np.linalg.LinAlgError:
        matrix = generate_invertible_matrix()
    
    return matrix

In [12]:
model_1  = LinearRegression()
model_1.fit(features_1, target)
prediction_1 = model_1.predict(features_1)
print(r2_score(target, prediction_1))

0.4249455028666683


**ВЫВОД** 

- Качество модели после переобучения практически не изменилось 

## Алгоритм преобразования

**Алгоритм**
Я немного запутался, вроде выше мы и так доказываем, что умножение на матрицу не повлияет на предсказание. Посмтрел рекомендации преподователя по проектам, он сказал, что можно сослаться на второй пункт.

**Обоснование**



## Проверка алгоритма

In [13]:
features_2=np.array([features])
features_3=np.rot90(features_2, 2)
target=list(reversed(target))

model_2=LinearRegression()
model_2.fit(features_3[0,:,:], target)
prediction_2=model_2.predict(features_3[0,:,:])
print(r2_score(target, prediction_2))

0.42494550286668


Опять же, качество модели практически не изменилось